In [1]:
import numpy as np
import jax
import jax.numpy as jnp
import haiku as hk

import sys
sys.path.append('../')

In [2]:
from neural_chess.models.policy_net import build_policy_net

In [3]:
# model config
model_config = {
    'vocab': 13,  # i.e) 12 pieces + empty square
    'embedding_dim': 128,
    'hidden_dim': 128,
    'head_dim': 32,
    'nb_layers': 4,
    'nb_heads': 4,
    'output_dim': 4096,
    'dropout': 0.1
}

In [4]:
# sample data
def get_dummy_batch(batch=8):
    """
    Generate a batch of dummy data, for the purpose of initialising the network parameter
    dict (note: this is not a legal board position!)
    """
    board_state = np.random.randint(0, 13, (batch, 64)).astype(np.int32)
    turn = np.random.binomial(p=0.5, n=1, size=(batch,)).astype(np.int32)
    castling_rights = np.random.binomial(p=0.5, n=1, size=(batch,)).astype(np.int32)
    en_passant = np.random.randint(0, 65, (batch,)).astype(np.int32)
    elo = np.random.random((batch,)).astype(np.float32)
    return board_state, turn, castling_rights, en_passant, elo

board_state, turn, castling_rights, en_passant, elo = get_dummy_batch(3)

print(board_state.shape)
print(elo.shape)
print(turn.shape)
print(castling_rights.shape)
print(en_passant.shape)

(3, 64)
(3,)
(3,)
(3,)
(3,)


In [5]:
# get a random key
key = jax.random.PRNGKey(42)

# initialise the network!
forward_fn = build_policy_net(**model_config)
init, apply = hk.transform(forward_fn)

params = init(key, board_state, turn, castling_rights, en_passant, elo, is_training=True)

In [6]:
# view the parameter tree
shapes = jax.tree_map(lambda x: x.shape, params)
print(shapes)

FlatMap({
  'board_emb': FlatMap({'embeddings': (13, 128)}),
  'board_pos_emb': FlatMap({'embeddings': (64, 128)}),
  'castle_emb': FlatMap({'embeddings': (2, 128)}),
  'elo_emb': FlatMap({'b': (128,), 'w': (1, 128)}),
  'en_passant_emb': FlatMap({'embeddings': (65, 128)}),
  'set_transformer/layer_norm': FlatMap({'offset': (128,), 'scale': (128,)}),
  'set_transformer/mlp/linear': FlatMap({'b': (256,), 'w': (128, 256)}),
  'set_transformer/mlp/linear_1': FlatMap({'b': (4096,), 'w': (256, 4096)}),
  'set_transformer/~_init_modules_for_layer/layer_norm': FlatMap({'offset': (128,), 'scale': (128,)}),
  'set_transformer/~_init_modules_for_layer/layer_norm_1': FlatMap({'offset': (128,), 'scale': (128,)}),
  'set_transformer/~_init_modules_for_layer/mlp/linear': FlatMap({'b': (256,), 'w': (128, 256)}),
  'set_transformer/~_init_modules_for_layer/mlp/linear_1': FlatMap({'b': (128,), 'w': (256, 128)}),
  'set_transformer/~_init_modules_for_layer/multi_head_attention/key': FlatMap({'b': (128,)

In [7]:
print(params['board_emb'])

FlatMap({
  'embeddings': DeviceArray([[-0.0178461 ,  0.00952877,  0.02462453, ...,  0.01094404,
                               0.01913168, -0.00570193],
                             [-0.02583723,  0.01335358,  0.02944723, ..., -0.00906039,
                               0.01591307,  0.00239701],
                             [-0.04328944,  0.01514734, -0.01018374, ..., -0.01745041,
                               0.01425417,  0.0224924 ],
                             ...,
                             [-0.03935785,  0.01419419, -0.01098523, ..., -0.01804546,
                               0.03128698,  0.01153706],
                             [ 0.00549005, -0.00130867, -0.03411865, ..., -0.00316766,
                               0.01009901, -0.0195477 ],
                             [ 0.00896102, -0.01897887, -0.00986449, ..., -0.01925972,
                               0.00177846, -0.04149825]], dtype=float32),
})


In [8]:
# perform a forward pass
output = apply(params, key, board_state, turn, castling_rights, en_passant, elo, is_training=True)